In [12]:
import folium
from pandas import DataFrame
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from math import radians, cos, sin, asin, sqrt


def haversine(p1, p2):
    
    lon1, lat1, lon2, lat2 = map(radians, [p1[1], p1[0], p2[1], p2[0]])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    return int(2 * asin(sqrt(sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2)) * 6371 * 1000)

data3t = {
        'lat':[34.8591,34.7819627101561,34.8069205283399,34.7864722047018], 
        'lon':[48.533,48.5231491280914,48.4910542253773,48.4866556882875]
    }
data = DataFrame(data3t)[['lat','lon']].to_numpy()

distances = np.zeros((data.shape[0],data.shape[0]))
for i in range(data.shape[0]):
    for j in range(data.shape[0]):
        if i!=j:
            distances[i][j] = haversine(data[i],data[j])
n_point = data.shape[0]
distances

array([[   0., 8624., 6951., 9116.],
       [8624.,    0., 4036., 3370.],
       [6951., 4036.,    0., 2308.],
       [9116., 3370., 2308.,    0.]])

In [13]:
import pyomo.environ as pyEnv
#Model
model = pyEnv.ConcreteModel()

#Indexes for the cities
model.M = pyEnv.RangeSet(n_point)                
model.N = pyEnv.RangeSet(n_point)

#Index for the dummy variable u
model.U = pyEnv.RangeSet(2,n_point)

In [14]:
#Decision variables xij
model.x = pyEnv.Var(model.N,model.M, within=pyEnv.Binary)

#Dummy variable ui
model.u = pyEnv.Var(model.N, within=pyEnv.NonNegativeIntegers,bounds=(0,n_point-1))

In [15]:
#Cost Matrix cij
model.c = pyEnv.Param(model.N, model.M,initialize=lambda model, i, j: distances[i-1][j-1])

In [16]:
def obj_func(model):
    return sum(model.x[i,j] * model.c[i,j] for i in model.N for j in model.M)

model.objective = pyEnv.Objective(rule=obj_func,sense=pyEnv.minimize)

In [17]:
def rule_const1(model,M):
    return sum(model.x[i,M] for i in model.N if i!=M ) == 1

model.const1 = pyEnv.Constraint(model.M,rule=rule_const1)

In [18]:
def rule_const2(model,N):
    return sum(model.x[N,j] for j in model.M if j!=N) == 1

model.rest2 = pyEnv.Constraint(model.N,rule=rule_const2)

In [19]:
def rule_const3(model,i,j):
    if i!=j: 
        return model.u[i] - model.u[j] + model.x[i,j] * n_point <= n_point-1
    else:
        #Yeah, this else doesn't say anything
        return model.u[i] - model.u[i] == 0 
    
model.rest3 = pyEnv.Constraint(model.U,model.N,rule=rule_const3)

In [20]:
#Prints the entire model
model.pprint()

3 Set Declarations
    c_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    N*M :   16 : {(1, 1), (1, 2), (1, 3), (1, 4), (2, 1), (2, 2), (2, 3), (2, 4), (3, 1), (3, 2), (3, 3), (3, 4), (4, 1), (4, 2), (4, 3), (4, 4)}
    rest3_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    U*N :   12 : {(2, 1), (2, 2), (2, 3), (2, 4), (3, 1), (3, 2), (3, 3), (3, 4), (4, 1), (4, 2), (4, 3), (4, 4)}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    N*M :   16 : {(1, 1), (1, 2), (1, 3), (1, 4), (2, 1), (2, 2), (2, 3), (2, 4), (3, 1), (3, 2), (3, 3), (3, 4), (4, 1), (4, 2), (4, 3), (4, 4)}

3 RangeSet Declarations
    M : Dimen=1, Size=4, Bounds=(1, 4)
        Key  : Finite : Members
        None :   True :   [1:4]
    N : Dimen=1, Size=4, Bounds=(1, 4)
        Key  : Finite : Members
        None :   True : 

In [27]:
#Solves
solver = pyEnv.SolverFactory('glpk')
result = solver.solve(model,tee = False)

#Prints the results
print(result)

    solver 'glpk'


ApplicationError: No executable found for solver 'glpk'

In [ ]:
l = list(model.x.keys())
for i in l:
    if model.x[i]() != 0:
        print(i,'--', model.x[i]())